In [15]:
import numpy as np
postingList=['my dog has flea problems help please',                #切分的词条
            'maybe not take him to dog park stupid',
             'my dalmation is so cute I love him',
             'stop posting stupid worthless garbage',
             'mr licks ate my steak how to stop him',
             'quit buying worthless dog food stupid']
classVec = np.array([0,1,0,1,0,1])    

from sklearn.feature_extraction.text import CountVectorizer
# 初始化CountVectorizer并进行文本特征提取
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(postingList)

# 显示特征向量和对应的单词
v=np.array(X.toarray())
print(v)
fn=np.array(vectorizer.get_feature_names())
print(fn)


#  P(Yj|X1,X2,X3) = P(Yj)  * (P(X1|Yj)*P(X2|Yj)*....P(Xn|Yj) / P(X1)*P(X2)*....P(Xn))
# P(Y1)假设是：侮辱类，先验概率就是0.5  P(Y2)是：非侮辱类，先验概率就是0.5 ，定义一个函数获取(P(X1|Yj)*P(X2|Yj)*....P(Xn|Yj)即可
#假设传入了一个语句，hello my name is za
# P（hello|Y1）就表示在侮辱类的数据行下 hello的概率
def pyj(text,classval):
    #将所有的文字按空格拆分出来 ["hello","my",...]
    wordArray=text.split(" ")
    probability=1
    #找到当前分类的索引，分类的索引相对于单词向量的行。
    indexArray=np.where(classVec==classval)
    #获取当前分类下的词条行。
    curClassArray=v[indexArray]
    for word in wordArray :
        wordIndex=np.where(fn==word)
        if len(wordIndex[0])==0:
            continue
        #获取当前特征列的词频数组。转换成1维。
        cu_fn=curClassArray[:,wordIndex[0][0]].flatten()
        #找到所有词频大于1/当前label下总的次数=p(x|y)
        pxy=len(np.where(cu_fn>0)[0])/len(cu_fn)
        probability*=pxy
    return probability
"""
  通过传入单词向量和分类结果训练数据集获取到每个特征在不同分类下的条件概率，以及对应分类的先验概率。
  利用贝叶斯分类器对文档进行分类时，要计算多个概率的乘积以获得文档属于某个类别的概率，
  即计算p(w0|1)p(w1|1)p(w2|1)。
  如果其中有一个概率值为0，那么最后的成绩也为0
  这样是不合理的，为了降低这种影响，可以将所有词的出现数初始化为1，并将分母初始化为2。这种做法就叫做拉普拉斯平滑(Laplace Smoothing)又被称为加1平滑，
  是比较常用的平滑方法，它就是为了解决0概率问题，具体参考拉普拉斯平滑目录。
"""
def trainData(vecList,classVec):
    #获取先验概率P(侮辱类),    P(非侮辱类)=1-P(侮辱类)
    PϹ侮辱类先验Ͻ=np.sum(classVec)/len(classVec)
    #找到所有classVec==0非侮辱类索引行并取得数据行。
    vec0=vecList[np.where(classVec==0)]
    #找到所有classVec==1侮辱类索引行并取得数据行。
    vec1=vecList[np.where(classVec==1)]
    #设置拉普拉斯平滑因子为1: ,分类的种类是2中所有分子+1，分母+2
    a=1
    #让分子都加上1
    vec0=np.add(vec0,a)
    vec1=np.add(vec1,a)
    #计算每个特征在对应分类下的条件概率，分母加上2
    PϹ特征l非侮辱类Ͻ=np.sum(vec0,axis=0)/(np.sum(vec0)+a*2)
    PϹ特征l侮辱类Ͻ=np.sum(vec1,axis=0)/(np.sum(vec1)+a*2)
    return PϹ特征l侮辱类Ͻ,PϹ特征l非侮辱类Ͻ,PϹ侮辱类先验Ͻ
'''
  注意求条件概率是找到对应的单词下在对应的分类下的乘积，比如
  you    are     a     dog       as               b
  0.001 0.0005  0.03   0.666     0.3             0.99
  
  传入的矩阵就是
  1       1      1      1        0               0
   
  实际条件侮辱类概率就是
  P(you|侮辱类)*P(are|侮辱类)*P(a|侮辱类)*P(dog|侮辱类)
  乘积小数位太多就可能导致小数位溢出，需要使用两个乘数的log来防止溢出
  log(P(you|侮辱类)*P(are|侮辱类)*P(a|侮辱类)*P(dog|侮辱类))=log(P(you|侮辱类))+log(P(are|侮辱类))+log(P(a|侮辱类))+log(P(dog|侮辱类))
  为了通过计算直接获取到对应的这些特征单词的和，可以先求出所有特征的log值和传入的矩阵乘积在求和就是上面的结果
'''
def classResult(wordVec,PϹ特征l侮辱类Ͻ,PϹ特征l非侮辱类Ͻ,PϹ侮辱类先验Ͻ):
    PϹ侮辱类Ͻ=np.sum(np.log(PϹ特征l侮辱类Ͻ)*wordVec)+np.log(PϹ侮辱类先验Ͻ)
    PϹ非侮辱类Ͻ=np.sum(np.log(PϹ特征l非侮辱类Ͻ)*wordVec)+np.log(1-PϹ侮辱类先验Ͻ)
    return 1 if PϹ侮辱类Ͻ>PϹ非侮辱类Ͻ else 0

PϹ特征l侮辱类Ͻ,PϹ特征l非侮辱类Ͻ,PϹ侮辱类先验Ͻ=(trainData(v,classVec))
print(PϹ特征l非侮辱类Ͻ,PϹ特征l侮辱类Ͻ,PϹ侮辱类先验Ͻ)


text=["you are a dog"]
testX = vectorizer.transform(text)

# 显示特征向量和对应的单词
testV=np.array(testX.toarray())
print(classResult(testV,PϹ特征l侮辱类Ͻ,PϹ特征l非侮辱类Ͻ,PϹ侮辱类先验Ͻ))

[[0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0]
 [0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1]]
['ate' 'buying' 'cute' 'dalmation' 'dog' 'flea' 'food' 'garbage' 'has'
 'help' 'him' 'how' 'is' 'licks' 'love' 'maybe' 'mr' 'my' 'not' 'park'
 'please' 'posting' 'problems' 'quit' 'so' 'steak' 'stop' 'stupid' 'take'
 'to' 'worthless']
[0.03389831 0.02542373 0.03389831 0.03389831 0.03389831 0.03389831
 0.02542373 0.02542373 0.03389831 0.03389831 0.04237288 0.03389831
 0.03389831 0.03389831 0.03389831 0.02542373 0.03389831 0.05084746
 0.02542373 0.02542373 0.03389831 0.02542373 0.03389831 0.02542373
 0.03389831 0.03389831 0.03389831 0.02542373 0.02542373 0.03389831
 0.02542373] [0.02631579 0.03508772 0.0